In [ ]:
# !pip3 install azure-cognitiveservices-speech

In [15]:
import os
import re
import azure.cognitiveservices.speech as speechsdk
import pandas as pd

# speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
speech_config = speechsdk.SpeechConfig(subscription='271cb26fab6e41c3a57ac3a93b354e03', region='eastus')
speech_config.speech_recognition_language="zh-CN"

def transcribe_Azure(filename):
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    speech_recognition_result = speech_recognizer.recognize_once_async().get()
    return speech_recognition_result.text

df_azure = pd.DataFrame({
    'Filename': [],
    'Start_time': [],
    'End_time': [],
    'Azure': []
})

In [17]:
for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationA/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Azure(f'segmented_audio/conversationA/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_azure.loc[len(df_azure)] = {'Filename': f'D00{number}_A','Start_time': start_time,'End_time': end_time,'Azure': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationB/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Azure(f'segmented_audio/conversationB/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_azure.loc[len(df_azure)] = {'Filename': f'D00{number}_B','Start_time': start_time,'End_time': end_time,'Azure': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/voice_command/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Azure(f'segmented_audio/voice_command/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_azure.loc[len(df_azure)] = {'Filename': f'P00{number}','Start_time': start_time,'End_time': end_time,'Azure': predicted}

df_azure.to_csv('predicted_transcription/Azure.csv')